<a href="https://colab.research.google.com/github/Devaraj-Sagar/ML2lab/blob/main/ML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def is_consistent(hypothesis, instance):
    """Check if an instance is consistent with a hypothesis."""
    for i in range(len(hypothesis)):
        if hypothesis[i] != '?' and hypothesis[i] != instance[i]:
            return False
    return True

def candidate_elimination(data):
    """
    Implements the Candidate-Elimination algorithm.
    """
    # Separate features (attributes) from the target
    attributes = data.iloc[:, :-1]
    target = data.iloc[:, -1]
    num_attributes = len(attributes.columns)

    # 1. Initialize S and G
    # Initialize G to the most general hypothesis
    G = [['?' for _ in range(num_attributes)]]

    # Initialize S with the first positive training example
    S = [None]
    for i, row in data.iterrows():
        if row.iloc[-1] == 'Yes':
            S = [list(row.iloc[:-1])]
            break

    print(f"Initialization:")
    print(f"S0: {S}")
    print(f"G0: {G}\n")

    # 2. Iterate through the training examples
    for i, row in data.iterrows():
        instance = list(row.iloc[:-1])
        label = row.iloc[-1]

        print(f"------------------- Instance {i+1} -------------------")
        print(f"Instance: {instance}, Label: {label}")

        if label == 'Yes': # Positive Example
            # Remove from G any hypothesis inconsistent with the instance
            G = [g for g in G if is_consistent(g, instance)]

            # Generalize S to be consistent with the new positive instance
            for s_index in range(len(S)):
                s = S[s_index]
                if not is_consistent(s, instance):
                    # Generalize s
                    for attr_index in range(num_attributes):
                        if s[attr_index] != instance[attr_index]:
                            S[s_index][attr_index] = '?'

            # Remove from S any hypothesis that is more general than another in S
            S = [s for s in S if not any(is_more_general(s, s_other) for s_other in S if s is not s_other)]

        else: # Negative Example
            # Remove from S any hypothesis consistent with the instance (should not happen in a noise-free dataset)
            S = [s for s in S if not is_consistent(s, instance)]

            # Specialize G to exclude the negative instance
            new_G = []
            for g in G:
                if not is_consistent(g, instance):
                    new_G.append(g) # Keep it if it already excludes the instance
                else:
                    # Create minimal specializations
                    for attr_index in range(num_attributes):
                        if g[attr_index] == '?':
                            # Get all possible values for this attribute
                            domain = attributes.iloc[:, attr_index].unique()
                            for value in domain:
                                if value != instance[attr_index]:
                                    new_g = list(g)
                                    new_g[attr_index] = value
                                    # Add specialization only if it's more general than some s in S
                                    if any(is_more_general(new_g, s) for s in S):
                                        new_G.append(new_g)

            G = new_G
            # Remove from G any hypothesis that is more specific than another in G
            G = [g for g in G if not any(is_more_general(g_other, g) for g_other in G if g is not g_other)]


        print(f"S{i+1}: {S}")
        print(f"G{i+1}: {G}\n")

        # If either boundary becomes empty, the version space has collapsed
        if not S or not G:
            print("Version space has collapsed. No consistent hypothesis found.")
            break

    return S, G

def is_more_general(h1, h2):
    """Check if hypothesis h1 is more general than or equal to h2."""
    more_general_parts = []
    for x, y in zip(h1, h2):
        mg = x == '?' or (x != '?' and x == y)
        more_general_parts.append(mg)
    return all(more_general_parts)

# --- Main Execution ---

# Create the dataset from the image
data = {
    'Age': ['Senior', 'Young', 'Young', 'Middle', 'Young', 'Senior'],
    'Income': ['Low', 'High', 'Medium', 'High', 'Low', 'Medium'],
    'Student': ['No', 'No', 'Yes', 'Yes', 'Yes', 'No'],
    'Credit_Rating': ['Fair', 'Fair', 'Excellent', 'Excellent', 'Fair', 'Fair'],
    'Employment': ['Unmployed', 'Employed', 'Employed', 'Employed', 'Unemployed', 'Employed'], # Correction based on image
    'Collateral': ['No', 'Yes', 'No', 'Yes', 'No', 'Yes'],
    'Loan_Approved': ['Yes', 'No', 'No', 'Yes', 'Yes', 'Yes']
}

# The first instance has 'Unemployed'. Let's correct the whole column based on the image.
data['Employment'] = ['Unemployed', 'Employed', 'Employed', 'Employed', 'Unemployed', 'Employed']
df = pd.DataFrame(data)

# Run the algorithm
final_S, final_G = candidate_elimination(df)

print("------------------- Final Result -------------------")
print("Final Specific Hypothesis (S):", final_S)
print("Final General Hypothesis (G):", final_G)

Initialization:
S0: [['Senior', 'Low', 'No', 'Fair', 'Unemployed', 'No']]
G0: [['?', '?', '?', '?', '?', '?']]

------------------- Instance 1 -------------------
Instance: ['Senior', 'Low', 'No', 'Fair', 'Unemployed', 'No'], Label: Yes
S1: [['Senior', 'Low', 'No', 'Fair', 'Unemployed', 'No']]
G1: [['?', '?', '?', '?', '?', '?']]

------------------- Instance 2 -------------------
Instance: ['Young', 'High', 'No', 'Fair', 'Employed', 'Yes'], Label: No
S2: [['Senior', 'Low', 'No', 'Fair', 'Unemployed', 'No']]
G2: [['Senior', '?', '?', '?', '?', '?'], ['?', 'Low', '?', '?', '?', '?'], ['?', '?', '?', '?', 'Unemployed', '?'], ['?', '?', '?', '?', '?', 'No']]

------------------- Instance 3 -------------------
Instance: ['Young', 'Medium', 'Yes', 'Excellent', 'Employed', 'No'], Label: No
S3: [['Senior', 'Low', 'No', 'Fair', 'Unemployed', 'No']]
G3: [['Senior', '?', '?', '?', '?', '?'], ['?', 'Low', '?', '?', '?', '?'], ['?', '?', '?', '?', 'Unemployed', '?'], ['?', '?', 'No', '?', '?', 'No